In [2]:
from __future__ import division

from scipy import optimize
import numpy as np
% matplotlib inline

### Deterministic Model

In [79]:
def deter_obj(x, q, c, e, f):
    """
        Objective function for deterministic problem. Single time horizon - no uncertainty.
        
        Parameters
        ----------
        x : {np.array}
            decision variables
        
        q : {np.array}
            sale price
        
        c : {np.array}
            purchase cost
        
        e : {np.array}
            inventory return
        
        f : {np.array}
            operation cost
    """
    s = x[:2]
    p = x[2:4]
    I = x[4:8]
    R = x[8:]
    
    obj = np.dot(q, s) - np.dot(c,p) + np.dot(e,I) - np.dot(f,R)
    
    return -1*obj

In [85]:
# sales price
q = np.array([100,80])

# purchase cost
c = np.array([10, 15])

# inventory coefficients
e = np.array([10, 15, 20, 25])

# operation cost
f = np.array([30, 35])

# decision variables - s3, s4, p1, p2, I1_f, I2_f, I3_f, I4_f, R1, R2

# bounds
bounds = [(20, 40), (20, 85), (20, 100), (30, 200), (0.0001, 20), (0.0001, 20), (0.0001, 20), (0.0001, 20), (20, 100), (30, 100)]

# init vals - boundary midpoints
x0 = np.array([(tup[1] - tup[0])/2 for tup in bounds])

cons = [
    {"type": "eq", "fun": lambda x: x[2] - 0.4*x[8] - x[4]},
    {"type": "eq", "fun": lambda x: x[3] - 0.6*x[8] - x[9] - x[5]},
    {"type": "eq", "fun": lambda x: 0.5*x[8] - x[6] - x[0]},
    {"type": "eq", "fun": lambda x: 0.6*x[9] - x[7] - x[1]}
]

# generate boundary constraints dynamically
dynamic_cons = []
for idx, bnd in enumerate(bounds):
    dynamic_cons.append({"type": "ineq", "fun": lambda x: x[idx] - bnd[0]})
    dynamic_cons.append({"type": "ineq", "fun": lambda x: bnd[1] - x[idx]})

cons += dynamic_cons

res = optimize.minimize(deter_obj, x0, args=(q,c,e,f), method="SLSQP", bounds=bounds, constraints=cons, options={"disp":True})
    
x = res.x

print "Sales Units - %s" % ", ".join(["S%s: %s" % (idx, val) for idx, val in enumerate(x[:2])])
print "Purchase Units - %s" % ", ".join(["P%s: %s" % (idx, val) for idx, val in enumerate(x[2:4])])
print "Inventory Units - %s" % ", ".join(["I%s_final: %s" % (idx, val) for idx, val in enumerate(x[4:8])])
print "Operation Levels - %s" % ", ".join(["R%s: %s" % (idx, val) for idx, val in enumerate(x[8:])])

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -493.320899987
            Iterations: 5
            Function evaluations: 61
            Gradient evaluations: 5
Sales Units - S0: 39.9999999997, S1: 20.0
Purchase Units - P0: 41.0000149999, P1: 88.1667849999
Inventory Units - I0_final: 8.99993500007, I1_final: 6.83316500007, I2_final: 0.000100000090872, I3_final: 0.00010000004724
Operation Levels - R0: 80.0001999996, R1: 33.3335000001


### TSSP - Without Risk

In [51]:
a = np.random.normal(0,1,25)

In [52]:
a

array([-0.44913704,  1.67834196,  1.55355153,  0.75830252, -0.432152  ,
        1.44588465, -0.51572166, -1.04472896,  0.79933831, -0.79047115,
        0.5338849 , -1.83893157, -0.51783001,  0.82696969,  1.06524289,
       -1.08051976, -0.62922052, -0.29300695,  0.01006503, -0.37225938,
       -0.50629288,  0.18850247,  0.14137509,  0.99053001,  1.62391023])

In [53]:
a = a.reshape(5,5)

In [54]:
a

array([[-0.44913704,  1.67834196,  1.55355153,  0.75830252, -0.432152  ],
       [ 1.44588465, -0.51572166, -1.04472896,  0.79933831, -0.79047115],
       [ 0.5338849 , -1.83893157, -0.51783001,  0.82696969,  1.06524289],
       [-1.08051976, -0.62922052, -0.29300695,  0.01006503, -0.37225938],
       [-0.50629288,  0.18850247,  0.14137509,  0.99053001,  1.62391023]])

In [56]:
a.diagonal()

array([-0.44913704, -0.51572166, -0.51783001,  0.01006503,  1.62391023])

In [50]:
a.T

array([[ 0.00681444, -0.32022964, -0.9002304 ,  0.12942348, -0.02298906],
       [ 1.16332388,  1.63940011,  2.30218522, -0.86326278, -0.26731533]])

In [37]:
a.shape

(10, 2)

In [44]:
np.dot([[1,2],[3,4]],[[1,1],])

array([3, 7])

In [32]:
np.dot(np.dot([[1,2],[3,4]],[1,1]), np.ones(2))

10.0

In [43]:
np.dot(np.random.normal(0,1,(10,2)), [3,4])

array([-2.53843056, -1.42678358, -5.87461745,  8.91204998, -4.84530833,
        2.78118296,  3.88662958, -2.56845004,  2.02103656,  1.46802793])

In [30]:
np.dot(np.dot(np.random.normal(0,1,(10,2)), [3,4]), np.ones(10))

-41.080388183994941

In [42]:
np.ones(10)*2.5

array([ 2.5,  2.5,  2.5,  2.5,  2.5,  2.5,  2.5,  2.5,  2.5,  2.5])

In [86]:
def tssp_obj(x, q, c, e, f, c_susl, num_scenarios):
    """
        Objective function for two stage stochastic problem.
        
        Parameters
        ----------
        x : {np.array}
            design variables
        
        q : {np.array}
            sales price (stochastic) - expected to have shape = (num_scenarios, num_det_var)
        
        c : {np.array}
            purchase costs
        
        e : {np.array}
            inventory returns
        
        f : {np.array}
            operation costs
        
        c_susl : {np.array}
            surplus and slack costs
        
        num_scenarios : {np.array}
            number of scenarios
    """
    scenario_prob = 1/num_scenarios
    
    # segment design variables
    num_stoch = q.size
    s = x[:num_stoch]
    s.reshape(num_scenarios,2)
    
    det_indices = [len(c), len(e), len(f)]
    for idx in range(0, len(det_indices)):
        if idx == 0:
            det_indices[idx] += num_stoch
            continue
        det_indices[idx] += num_stoch + det_indices[idx-1]
    p = x[num_stoch:det_indices[0]]
    I = x[det_indices[0]:det_indices[1]]
    R = x[det_indices[1]:det_indices[2]]
    
    su_sl = x[det_indices[2]:]
    su_sl.reshape(num_scenarios, 2)
    
    # <det> + scenario_prob*(sum_s <stoch>) - (su_sl_cost*<su_sl>)
    det_seg = -np.dot(c,p) + np.dot(e,I) - np.dot(f,R)
    
    # Sx2 * 2xS - np.dot is matrix multiplication for 2-D arrays
    stoch_seg = np.dot(q,s.T)
    stoch_seg = np.dot(stoch_seg.diagonal(),np.ones(num_scenarios)*scenario_prob)
    
    susl_seg = np.dot(np.dot(su_sl, c_susl), np.ones(num_scenarios)*scenario_prob)
    
    obj = stoch_seg + det_seg - susl_seg
    
    return -1*obj

In [ ]:
# num_scenarios = 5

# surplus, slack penalties
csusl = np.array([20,20])

# sales price samples - num_scenarios*num_stoch_var
q_samples = np.random.normal(90,27,(num_scenarios, 2))

# purchase cost
c = np.array([10, 15])

# inventory coefficients
e = np.array([10, 15, 20, 25])

# operation cost
f = np.array([30, 35])

# bounds
# <det> + <sampled items> + <su/sl => 2*num_scenarios>
bounds = [(20, 40), (20, 85)]*num_scenarios # bounds unchanged for stochastic vars
bounds += [(20, 100), (30, 200), (0, 20), (0, 20), (0, 20), (0, 20), (20, 100), (30, 100)]
bounds += [(0,None),(0,None)]*num_scenarios # su/sl bounds

# init vals
x0 = np.array([(tup[1] - tup[0])/2 for tup in bounds])

# dynamically generate equality constraints
cons = [
    {"type": "eq", "fun": lambda x: x[2] - 0.4*x[8] - x[4]},
    {"type": "eq", "fun": lambda x: x[3] - 0.6*x[8] - x[9] - x[5]},
    {"type": "eq", "fun": lambda x: 0.5*x[8] - x[6] - x[0]},
    {"type": "eq", "fun": lambda x: 0.6*x[9] - x[7] - x[1]}
]

cons = []
# last index of stochastic variables in decision array (sei)
sei = q_samples.size - 1
# starting index of surplus and slack variables in decision array (ssi)
ssi = len(bounds) - (2*num_scenarios + 1)
for idx in range(0,num_scenarios):
    jump_indices = (2*idx, 2*idx+1)
    cons.append({"type":"eq", "fun": lambda x: x[sei+1] - 0.4*x[sei+7] - x[sei+3] + x[ssi+jump_indices[0]] - x[ssi+jump_indices[1]]})
    cons.append({"type": "eq", "fun": lambda x: x[sei+2] - 0.6*x[sei+7] - x[sei+8] - x[sei+4] + x[]})


### TSSP - Financial Risk

### RO SP FR

### Mean Markovitz Model

In [6]:
def risk_const(x, params, risk_param, profit_lb):
    """
        Risk constraint for Mean-Markovitz model.
        
        Parameters
        ----------
        x : {np.array}
            array of decision variables
            
        params : {dict}
            dictionary of parameters used for constraint generation
        
        risk_param : {float}
            custom risk parameter for variation penalty
        
        profit_lb : {float}
            lower bound for profit (t)
            
        Parameter Dictionary
        --------------------
        sales_mean : {np.array}
            mean coefficients for sales variable (selling price)
        
        sales_std : {np.array}
            standard deviation for sales variables
            
        purchase_coeff : {np.array}
            purchase cost
        
        inventory_coeff : {np.array}
            positive inventory returns
        
        operation_coeff : {np.array}
            operation costs
    """
    mu_q = params["sales_mean"]
    std_q = params["sales_std"]
    c = params["purchase_coeff"]
    e = params["inventory_coeff"]
    f = params["operation_coeff"]
    
    # segment decision variables
    s = x[:2]
    p = x[2:4]
    I = x[4:8]
    R = x[8:]
    
    const = np.dot(mu_q, s) - np.dot(c,p) + np.dot(e,I) - np.dot(f,R) - \
        risk_param*np.sqrt(np.dot(np.square(std_q),np.square(y))) - profit_lb
    
    return const

In [ ]:
# profit lb
t = 10000

# risk param
theta = 0.3

# mean sales price
mean_q = np.array([90, 80])

# std sales price
std_q = np.array([27, 20])

# purchase cost
c = np.array([10, 15])

# inventory coefficients
e = np.array([10, 15, 20, 25])

# operation cost
f = np.array([30, 35])

# decision variables - s3, s4, p1, p2, I1_f, I2_f, I3_f, I4_f, R1, R2

# bounds
bounds = [(20, 40), (20, 85), (20, 100), (30, 200), (0.0001, 20), (0.0001, 20), (0.0001, 20), (0.0001, 20), (20, 100), (30, 100)]

# init vals - boundary midpoints
x0 = np.array([(tup[1] - tup[0])/2 for tup in bounds])

# maximize profit
obj = lambda x: -t

params = {"sales_mean":mean_q, "sales_std": std_q, "purchase_coeff": c, "inventory_coeff": e,
          "operation_coeff": f}

# constraints
cons = (
    {"type": "ineq", "fun": risk_const, "args": (params,theta,t)},
    {"type": "eq", "fun": lambda x: x[2] - 0.4*x[8] - x[4]},
    {"type": "eq", "fun": lambda x: x[3] - 0.6*x[8] - x[9] - x[5]},
    {"type": "eq", "fun": lambda x: 0.5*x[8] - x[6] - x[0]},
    {"type": "eq", "fun": lambda x: 0.6*x[9] - x[7] - x[1]}
)

res = optimize.minimize(obj, x0, bounds=bounds, constraints=cons, options={"disp": True})

x = res.x

print "Sales Units - %s" % ", ".join(["S%s: %s" % (idx, val) for idx, val in enumerate(x[:2])])
print "Purchase Units - %s" % ", ".join(["P%s: %s" % (idx, val) for idx, val in enumerate(x[2:4])])
print "Inventory Units - %s" % ", ".join(["I%s_final: %s" % (idx, val) for idx, val in enumerate(x[4:8])])
print "Operation Levels - %s" % ", ".join(["R%s: %s" % (idx, val) for idx, val in enumerate(x[8:])])